# Input and one Dense Layer, own weights

### Load the model

In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"
model = tf.keras.models.load_model(model_path, custom_objects=co)

2025-08-25 11:42:04.607068: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-25 11:42:06.372935: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-25 11:42:06.372979: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-25 11:42:06.379196: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-25 11:42:07.443911: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-25 11:42:07.446444: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Prepare data for tests

In [2]:
import numpy as np

In [3]:
input_quantizer = quantized_bits(14, 5, alpha=1)
output_quantizer = quantized_bits(18,13,alpha=1)
bias_quantizer = quantized_bits(18,12,alpha=1)

In [ ]:
X = 

X_flat = X.reshape(X.shape[0], -1)
X_flat = input_quantizer(X_flat)
X_flat.dtype

tf.float32

### Fragmentary model

In [9]:
fragm_model = tf.keras.Model(inputs=model.layers[0].input, outputs=model.layers[1].output, name='fragm_model')
fragm_model.summary()

Model: "fragm_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 57)]              0         
                                                                 
 q_dense (QDense)            (None, 29)                1682      
                                                                 
Total params: 1682 (6.57 KB)
Trainable params: 1682 (6.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Conversion into a hls4ml model

In [10]:
import hls4ml

hls_config = hls4ml.utils.config_from_keras_model(
    fragm_model,
    granularity='name', 
    default_precision='fixed<14,7>',
    backend='Vitis')
print("-----------------------------------")


/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 57]], output shape: [None, 57]
Layer name: q_dense, layer type: QDense, input shapes: [[None, 57]], output shape: [None, 29]
-----------------------------------


/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [11]:
hls_model = hls4ml.converters.convert_from_keras_model(
    fragm_model, hls_config=hls_config, backend='Vitis',
    output_dir='./hls_fragm_0_1'
)

Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 57]], output shape: [None, 57]
Layer name: q_dense, layer type: QDense, input shapes: [[None, 57]], output shape: [None, 29]
Creating HLS model


In [12]:
hls_model.compile()

Writing HLS project


/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done


### Compare predictions for the QKeras and HLS model

In [126]:
X = numpy.full((2048,57), -5.554452 ,dtype=float)
X = input_quantizer(X)
X = X.numpy().reshape(X.shape[0], -1)
X[:10]

array([[-5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875],
       [-5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546875, -5.5546875, -5.5546875, -5.5546875, -5.5546875,
        -5.5546

In [127]:
pred_qkeras = fragm_model.predict(X)
pred_hls = hls_model.predict(numpy.ascontiguousarray(X))

64/64 [==============================] - 0s 791us/step


Both shapes (2048,29) and dtypes (float32) consistent

Differences are mostly 0.0625 (0.0001)$_2$, found also 0.125 (0.001)$_2$

In [130]:
pred_qkeras[:10]

array([[ 0.    ,  8.3125,  6.25  ,  0.    ,  5.8125,  7.3125,  9.    ,
         0.    ,  0.    ,  3.75  ,  9.    ,  1.9375, 13.3125,  0.    ,
         0.    ,  0.    ,  5.6875,  4.1875,  0.    , 21.5   , 20.125 ,
         0.    , 17.5   ,  0.    ,  0.    , 14.6875, 12.1875,  6.375 ,
         0.5625],
       [ 0.    ,  8.3125,  6.25  ,  0.    ,  5.8125,  7.3125,  9.    ,
         0.    ,  0.    ,  3.75  ,  9.    ,  1.9375, 13.3125,  0.    ,
         0.    ,  0.    ,  5.6875,  4.1875,  0.    , 21.5   , 20.125 ,
         0.    , 17.5   ,  0.    ,  0.    , 14.6875, 12.1875,  6.375 ,
         0.5625],
       [ 0.    ,  8.3125,  6.25  ,  0.    ,  5.8125,  7.3125,  9.    ,
         0.    ,  0.    ,  3.75  ,  9.    ,  1.9375, 13.3125,  0.    ,
         0.    ,  0.    ,  5.6875,  4.1875,  0.    , 21.5   , 20.125 ,
         0.    , 17.5   ,  0.    ,  0.    , 14.6875, 12.1875,  6.375 ,
         0.5625],
       [ 0.    ,  8.3125,  6.25  ,  0.    ,  5.8125,  7.3125,  9.    ,
         0.    ,  0.   

In [129]:
i = numpy.random.randint(0,2048, 1)
(pred_qkeras[i]-pred_hls[i])

array([[0.    , 0.1875, 0.125 , 0.    , 0.125 , 0.125 , 0.125 , 0.    ,
        0.    , 0.1875, 0.125 , 0.125 , 0.1875, 0.    , 0.    , 0.    ,
        0.1875, 0.1875, 0.    , 0.1875, 0.125 , 0.    , 0.1875, 0.    ,
        0.    , 0.1875, 0.1875, 0.1875, 0.125 ]], dtype=float32)